## Prediction of store sales

#### Importing Libraries

In [28]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from src.utils import check_missing_data

#### Loading Data

In [29]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_store = pd.read_csv('../data/store.csv')

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_30128\267565919.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../data/train.csv')


#### Data Preprocessing 

##### Handling Missing Values

In [30]:
# checking for missing values
missing_data_df_store = check_missing_data(df_store)
print(missing_data_df_store)

                 Column Name  Missing Values  Percentage Missing
3        CompetitionDistance               3            0.269058
4  CompetitionOpenSinceMonth             354           31.748879
5   CompetitionOpenSinceYear             354           31.748879
7            Promo2SinceWeek             544           48.789238
8            Promo2SinceYear             544           48.789238
9              PromoInterval             544           48.789238


In [31]:
# Handle missing values
df_store['CompetitionDistance'].fillna(df_store['CompetitionDistance'].median(), inplace=True)
df_store['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
df_store['CompetitionOpenSinceYear'].fillna(0, inplace=True)
df_store['Promo2SinceWeek'].fillna(0, inplace=True)
df_store['Promo2SinceYear'].fillna(0, inplace=True)
df_store['PromoInterval'].fillna('', inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_30128\2325696550.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_store['CompetitionDistance'].fillna(df_store['CompetitionDistance'].median(), inplace=True)
C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_30128\2325696550.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obje

In [32]:
# checking for missing values
missing_data_df_store = check_missing_data(df_store)
print(missing_data_df_store)

Success: No missing values.


In [33]:
# Remove leading and trailing whitespaces
df_train['StateHoliday'] = df_train['StateHoliday'].str.strip()

In [34]:
# checking for missing values
missing_data_df_train = check_missing_data(df_train)
print(missing_data_df_train)

    Column Name  Missing Values  Percentage Missing
7  StateHoliday          131072           12.885454


In [35]:
# Handle missing values
df_train['StateHoliday'].fillna(0, inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_30128\1216057678.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['StateHoliday'].fillna(0, inplace=True)


In [36]:
# checking for missing values
missing_data_df_train = check_missing_data(df_train)
print(missing_data_df_train)

Success: No missing values.


In [37]:
# checking for missing values
missing_data_df_test = check_missing_data(df_test)
print(missing_data_df_test)

  Column Name  Missing Values  Percentage Missing
4        Open              11            0.026772


In [38]:
# Handle missing values
df_test['Open'].fillna(0, inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_30128\3366596396.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['Open'].fillna(0, inplace=True)


In [39]:
# checking for missing values
missing_data_df_test = check_missing_data(df_test)
print(missing_data_df_test)

Success: No missing values.
